<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 3.2.1 
# *Querying the International Space Station*

## The OpenNotify API

The OpenNotify API exposes a few attributes of the International Space Station (ISS) via a simple, authentication-free interface. The simplicity of this API precludes any need for a dedicated Python library. However, as with many APIs, it accepts requests according to HTTP standards and returns responses in JSON format, so the Python libraries request and json will make managing the I/O simpler still.

In [5]:
import requests
import json
from datetime import datetime, date, time

This request fetches the latest position of the international space station:

In [6]:
response = requests.get("http://api.open-notify.org/iss-now.json")

Print the status code of the response:

In [7]:
print(response.status_code)

200


We can also request the current position of the ISS and the next few times at which it will be over a certain location. The latitude and longitude of Sydney are (-33.87, 151.21). Create a dict named `parameters` whose elements are key:value pairs named `lat` and `lon`, containing those figures:

In [8]:
parameters = {"lat": -33.87, "lon": 151.21}

The API request we want starts like the last one, but the last part of the endpoint is "iss-pass.json", and it takes a second argument -- namely, the `parameters` value created above. Compose and execute the API request:

In [9]:
response2 = requests.get("http://api.open-notify.org/iss-pass.json", params=parameters)

Print the response header:

In [10]:
print(response2.headers)

{'Server': 'nginx/1.10.3', 'Date': 'Mon, 31 May 2021 10:13:36 GMT', 'Content-Type': 'application/json', 'Content-Length': '521', 'Connection': 'keep-alive', 'Via': '1.1 vegur'}


Print the content of the response (the data that the server returned):

In [11]:
content = response2.content
print(content)

b'{\n  "message": "success", \n  "request": {\n    "altitude": 100, \n    "datetime": 1622456016, \n    "latitude": -33.87, \n    "longitude": 151.21, \n    "passes": 5\n  }, \n  "response": [\n    {\n      "duration": 570, \n      "risetime": 1622471438\n    }, \n    {\n      "duration": 655, \n      "risetime": 1622477187\n    }, \n    {\n      "duration": 502, \n      "risetime": 1622483116\n    }, \n    {\n      "duration": 346, \n      "risetime": 1622489101\n    }, \n    {\n      "duration": 484, \n      "risetime": 1622494941\n    }\n  ]\n}\n'


Note that this is a Python byte string:

In [12]:
# print the type
print(type(content))

<class 'bytes'>


Print just the "content-type" value from the header:

In [13]:
print(response2.headers['Content-Type'])

application/json


JSON was designed to be easy for computers to read, not for people. The `requests` library can decode the JSON byte string:

In [14]:
overheads = response2.json()
print(overheads)

{'message': 'success', 'request': {'altitude': 100, 'datetime': 1622456016, 'latitude': -33.87, 'longitude': 151.21, 'passes': 5}, 'response': [{'duration': 570, 'risetime': 1622471438}, {'duration': 655, 'risetime': 1622477187}, {'duration': 502, 'risetime': 1622483116}, {'duration': 346, 'risetime': 1622489101}, {'duration': 484, 'risetime': 1622494941}]}


What kind (type) of object did this give us?

In [15]:
type(overheads)

dict

Python dicts {key : value} are easier to work with, but the data we want is still buried in that data structure, so we have to dig it out. First, extract the `response` value to a separate dict:

In [16]:
passes = dict(response2.json())
passes

{'message': 'success',
 'request': {'altitude': 100,
  'datetime': 1622456016,
  'latitude': -33.87,
  'longitude': 151.21,
  'passes': 5},
 'response': [{'duration': 570, 'risetime': 1622471438},
  {'duration': 655, 'risetime': 1622477187},
  {'duration': 502, 'risetime': 1622483116},
  {'duration': 346, 'risetime': 1622489101},
  {'duration': 484, 'risetime': 1622494941}]}

Now extract the `risetime` strings into an array called `srisetimes`:

In [17]:
risetime_list = passes['response']

srisetimes = []
for i in risetime_list:
    srisetimes.append(i['risetime'])

print('risetime: ', srisetimes)

risetime:  [1622471438, 1622477187, 1622483116, 1622489101, 1622494941]


These values are in Unix Timestamp format.  Convert these to an array of Python `datetime` values called `risetimes`:

```
datetime.fromtimestamp(str)
```

In [18]:
risetimes = []

for i in srisetimes:
    risetimes.append(datetime.fromtimestamp(i))

risetimes

[datetime.datetime(2021, 5, 31, 22, 30, 38),
 datetime.datetime(2021, 6, 1, 0, 6, 27),
 datetime.datetime(2021, 6, 1, 1, 45, 16),
 datetime.datetime(2021, 6, 1, 3, 25, 1),
 datetime.datetime(2021, 6, 1, 5, 2, 21)]

Finally, print these in a format that people understand:



```
str.strftime('%d/%m/%y %I:%M')

27/03/19 10:59
27/03/19 12:35
27/03/19 02:11
28/03/19 03:42
28/03/19 05:13
```



In [19]:
for risetime in risetimes:
    print(risetime.strftime('%d/%m/%y %I:%M'))

31/05/21 10:30
01/06/21 12:06
01/06/21 01:45
01/06/21 03:25
01/06/21 05:02


Here is an endpoint that tells us who is onboard:

In [20]:
response3 = requests.get("http://api.open-notify.org/astros.json")

In [27]:
response3.headers

{'Server': 'nginx/1.10.3', 'Date': 'Mon, 31 May 2021 10:13:37 GMT', 'Content-Type': 'application/json', 'Content-Length': '355', 'Connection': 'keep-alive', 'access-control-allow-origin': '*'}

In [25]:
response3.text

'{"number": 7, "message": "success", "people": [{"name": "Mark Vande Hei", "craft": "ISS"}, {"name": "Oleg Novitskiy", "craft": "ISS"}, {"name": "Pyotr Dubrov", "craft": "ISS"}, {"name": "Thomas Pesquet", "craft": "ISS"}, {"name": "Megan McArthur", "craft": "ISS"}, {"name": "Shane Kimbrough", "craft": "ISS"}, {"name": "Akihiko Hoshide", "craft": "ISS"}]}'

In [26]:
response3.content

b'{"number": 7, "message": "success", "people": [{"name": "Mark Vande Hei", "craft": "ISS"}, {"name": "Oleg Novitskiy", "craft": "ISS"}, {"name": "Pyotr Dubrov", "craft": "ISS"}, {"name": "Thomas Pesquet", "craft": "ISS"}, {"name": "Megan McArthur", "craft": "ISS"}, {"name": "Shane Kimbrough", "craft": "ISS"}, {"name": "Akihiko Hoshide", "craft": "ISS"}]}'

Referring to the methods used above, extract the number of astronauts and their names:

In [28]:
print(response3.status_code)
print(response3.headers)
data = dict(response3.json())
print('number: ', data['number'])

people = data['people']
names = []
for i in people:
    names.append(i['name'])

for name in names:
    print(name)

200
{'Server': 'nginx/1.10.3', 'Date': 'Mon, 31 May 2021 10:13:37 GMT', 'Content-Type': 'application/json', 'Content-Length': '355', 'Connection': 'keep-alive', 'access-control-allow-origin': '*'}
number:  7
Mark Vande Hei
Oleg Novitskiy
Pyotr Dubrov
Thomas Pesquet
Megan McArthur
Shane Kimbrough
Akihiko Hoshide


## HOMEWORK


1. Write a simple handler for the response status code (refer to lab resources slide for HTTP response codes). As this Jupyter Notebook is an interactive device, the handler does not need to manage subsequent code execution (i.e. by branching or aborting execution), although it should return something that could be used to do so if deployed in a Python program.

In [29]:
#ANSWER:
def handleResponse(response, verbose = False):
    '''
    Returns Boolean Value, Status Code, 
    '''
  # if Status Code is 200 return false, and status code
    if(response.status_code == 200):
        return False, response.status_code
  # Otherwise Return True and Status Code
    else:
        return True, response.status_code

2. Test your response handler on some correct and incorrect API calls.

In [30]:
response = requests.get("http://api.open-notify.org/astros.json")
if handleResponse(response)[0]:
    print('API call failed. Resolve issue before continuing!')
    
response = requests.get("http://api.open-notify.org/iss-pass.json")
handleResponse(response, True)[0]

True

>

>

>



---



---



> > > > > > > > > © 2021 Institute of Data


---



---



